In [91]:
import pandas as pd                               # panda
import pandas_datareader as pdr                   # Baixar dados yahoo

import matplotlib.pyplot as plt                   # grafico
import matplotlib.dates as mdates                 # trabalhar datas

import statsmodels.api as sm                      # regressao
from statsmodels.api import add_constant          # para a regressao considerar o intercepto

import numpy as np
import scipy.stats
#import seaborn as sns

from statsmodels.tsa.stattools import coint       # Funcao Coint ()
from statsmodels.tsa.stattools import adfuller

# Declaracoes
pd.options.display.float_format = '{:.2f}'.format

#!pip install MetaTrader5
#!pip install --upgrade MetaTrader5
import MetaTrader5 as mt5
from datetime import datetime
from datetime import date
import time
import pytz

import ipywidgets

%matplotlib inline

In [63]:
#time_stp = time.mktime(datetime.strptime(time_trade,"%Y-%m-%d-%H-%M").timetuple())
#dt_object = datetime.fromtimestamp(timestamp)

In [64]:
print ('Current date/time: {}'.format(datetime.now()))

Current date/time: 2020-08-06 15:11:21.958948


In [65]:
timeobj = datetime.now()
tempo = str(timeobj)

In [66]:
time_trade = tempo[:-10]
time_trade

'2020-08-06 15:11'

In [67]:
a = time.mktime(datetime.strptime(time_trade,"%Y-%m-%d %H:%M").timetuple())

In [102]:
b = datetime.fromtimestamp(a)
b

datetime.datetime(2020, 8, 6, 15, 11)

In [56]:
import time
import datetime
s = "01/12/2011"
time.mktime(datetime.datetime.strptime(s, "%d/%m/%Y").timetuple())

1322708400.0

In [119]:
def Get_data(tickers,inicio,fim,janela):

    # Checa se ja estamos conectados, se não, conecta
    if (mt5.terminal_info()==None):
          
        # conecte-se ao MetaTrader 5
        if not mt5.initialize():
            print("initialize() failed, error code =",mt5.last_error())
            mt5.shutdown()
        
    #quebra as strings dates(inicio,fim) recebidas e transformando em integer
    inicio_= inicio.split('-')
    inicio = [int(element) for element in inicio_]
    fim_= fim.split('-')
    fim = [int(element) for element in fim_]

    timezone = pytz.timezone("Etc/UTC")
    
    #Verifica caso seja timeframe diario ou intraday
    if janela == '1d':
        dt_inicio = datetime(inicio[0], inicio[1], inicio[2], tzinfo=timezone)
        dt_fim = datetime(fim[0], fim[1], fim[2], tzinfo=timezone)
        timeframe = mt5.TIMEFRAME_D1

    else:
        dt_inicio = datetime(inicio[0], inicio[1], inicio[2], inicio[3], inicio[4], tzinfo=timezone)
        dt_fim = datetime(fim[0], fim[1], fim[2], fim[3], fim[4], tzinfo=timezone)
        timeframe = mt5.TIMEFRAME_M5
        
    # obtendo o par de acoes com base nas datas enviadas nas strings, definindo o timezone para nao interfir com o tisp do mt5
    if (len (tickers) == 1):        
        stock1 = mt5.copy_rates_range(tickers[0], timeframe, dt_inicio, dt_fim)    
        stock_df = pd.DataFrame(stock1)[['close','time']].rename(columns={'close': tickers[0]})
                
    else:    
        stock1 = mt5.copy_rates_range(tickers[0], timeframe, dt_inicio, dt_fim)
        stock2 = mt5.copy_rates_range(tickers[1], timeframe, dt_inicio, dt_fim)
        esquerda = pd.DataFrame(stock1)[['close','time']].rename(columns={'close': tickers[0]})
        direita = pd.DataFrame(stock2)[['close','time']].rename(columns={'close': tickers[1]})
        stock_df = pd.merge(left=esquerda, right=direita, left_on='time', right_on='time')
    
    if (janela != '1d'):
        stock_df['Date'] = stock_df['time'].apply(lambda x: str (datetime.fromtimestamp(x, tz=timezone))[:-9])
    else:
        stock_df['Date'] = stock_df['time'].apply(lambda x: date.fromtimestamp(x+21600)) #adicionando 21600 pra corrigir o fuso no momento de converter tsp em str
        stock_df.drop('time',axis=1,inplace = True)
        
    return stock_df.set_index('Date')

In [129]:
ticker =    ['PETR4','BBAS3']
ticker_a =  ['PETR4']

In [130]:
Get_data(ticker, '2020-07-20-11-00','2020-07-20-11-55','M5')

,PETR4,time,BBAS3
Date,,,
2020-07-20 11:00,22.58,1595242800,34.97
2020-07-20 11:05,22.62,1595243100,35.01
2020-07-20 11:10,22.59,1595243400,35.02
2020-07-20 11:15,22.65,1595243700,35.10
2020-07-20 11:20,22.72,1595244000,35.08
2020-07-20 11:25,22.71,1595244300,35.10
2020-07-20 11:30,22.73,1595244600,35.19
2020-07-20 11:35,22.79,1595244900,35.29
2020-07-20 11:40,22.80,1595245200,35.33


In [131]:
Get_data(ticker, '2020-07-20','2020-07-30','1d')

,PETR4,BBAS3
Date,,
2020-07-20,22.74,35.32
2020-07-21,23.37,35.49
2020-07-22,23.05,35.20
2020-07-23,22.57,34.35
2020-07-24,22.73,33.95
2020-07-27,23.20,34.91
2020-07-28,22.80,35.03
2020-07-29,23.17,35.95
2020-07-30,22.82,34.80


In [132]:
Get_data(ticker_a, '2020-07-20','2020-07-30','1d')

,PETR4
Date,
2020-07-20,22.74
2020-07-21,23.37
2020-07-22,23.05
2020-07-23,22.57
2020-07-24,22.73
2020-07-27,23.20
2020-07-28,22.80
2020-07-29,23.17
2020-07-30,22.82


In [137]:
Get_data(ticker_a, '2020-07-20-10-00','2020-07-30-12-00','M5')

,PETR4,time
Date,,
2020-07-20 10:10,22.65,1595239800
2020-07-20 10:15,22.63,1595240100
2020-07-20 10:20,22.60,1595240400
2020-07-20 10:25,22.59,1595240700
2020-07-20 10:30,22.65,1595241000
...,...,...
2020-07-30 11:40,22.55,1596109200
2020-07-30 11:45,22.59,1596109500
2020-07-30 11:50,22.60,1596109800
